# COW vs MOR 직접 비교 실험

동일한 데이터와 동일한 작업을 COW/MOR 테이블에서 각각 수행하고,  
**파일 수, 크기, 쓰기/읽기 시간**을 정량적으로 비교합니다.

## 실험 설계

| 단계 | 작업 | 설명 |
|------|------|------|
| 1 | INSERT | 200건 초기 데이터 적재 |
| 2 | UPDATE | 50건 상태 변경 |
| 3 | DELETE | 30건 삭제 |
| 4 | SELECT | 전체 읽기 |

동일한 시나리오를 COW와 MOR 양쪽에서 실행하여 차이를 관찰합니다.

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

import time
import pandas as pd

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [ ]:
spark = create_spark_session()

COW_TABLE = "demo.lab.cmp_cow_orders"
MOR_TABLE = "demo.lab.cmp_mor_orders"
COW_PATH = "/home/jovyan/data/warehouse/lab/cmp_cow_orders"
MOR_PATH = "/home/jovyan/data/warehouse/lab/cmp_mor_orders"

## 테이블 생성

동일한 스키마, 동일한 파티셔닝 — **TBLPROPERTIES만 다릅니다**.

In [ ]:
# 기존 테이블 정리
spark.sql(f"DROP TABLE IF EXISTS {COW_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {MOR_TABLE}")

# COW 테이블
spark.sql(f"""
CREATE TABLE {COW_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
""")
print(f"COW 테이블 생성 완료: {COW_TABLE}")

# MOR 테이블
spark.sql(f"""
CREATE TABLE {MOR_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
)
""")
print(f"MOR 테이블 생성 완료: {MOR_TABLE}")

---
## 단계 1: INSERT 200건

동일한 데이터(seed=42)를 양쪽 테이블에 삽입합니다.

In [ ]:
# 동일한 데이터 생성
orders = generate_orders(num_records=200, seed=42)
df = to_spark_df(spark, orders)
df.cache()  # 동일 데이터를 두 번 쓰므로 캐시

# COW INSERT
start = time.time()
df.writeTo(COW_TABLE).append()
cow_insert_time = time.time() - start
print(f"COW INSERT 시간: {cow_insert_time:.3f}초")

# MOR INSERT
start = time.time()
df.writeTo(MOR_TABLE).append()
mor_insert_time = time.time() - start
print(f"MOR INSERT 시간: {mor_insert_time:.3f}초")

df.unpersist()
print(f"\n양쪽 테이블 모두 {spark.sql(f'SELECT COUNT(*) FROM {COW_TABLE}').collect()[0][0]}건 삽입 완료")

### 관찰 포인트 — INSERT

INSERT는 COW/MOR 차이가 없습니다. 둘 다 새 데이터 파일을 생성할 뿐입니다.

---
## 단계 2: UPDATE 50건

order_id 기준으로 처음 50건의 상태를 변경합니다.

In [ ]:
# COW UPDATE
start = time.time()
spark.sql(f"UPDATE {COW_TABLE} SET status = 'refunded' WHERE order_id <= 50")
cow_update_time = time.time() - start
print(f"COW UPDATE 시간: {cow_update_time:.3f}초")

# MOR UPDATE
start = time.time()
spark.sql(f"UPDATE {MOR_TABLE} SET status = 'refunded' WHERE order_id <= 50")
mor_update_time = time.time() - start
print(f"MOR UPDATE 시간: {mor_update_time:.3f}초")

ratio = cow_update_time / mor_update_time if mor_update_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x")

### 관찰 포인트 — UPDATE

- COW: 영향받은 파티션의 데이터 파일을 **통째로 재작성** → 느림
- MOR: Delete File + 작은 데이터 파일만 추가 → 빠름

---
## 단계 3: DELETE 30건

order_id 기준으로 51~80번 주문을 삭제합니다.

In [ ]:
# COW DELETE
start = time.time()
spark.sql(f"DELETE FROM {COW_TABLE} WHERE order_id BETWEEN 51 AND 80")
cow_delete_time = time.time() - start
print(f"COW DELETE 시간: {cow_delete_time:.3f}초")

# MOR DELETE
start = time.time()
spark.sql(f"DELETE FROM {MOR_TABLE} WHERE order_id BETWEEN 51 AND 80")
mor_delete_time = time.time() - start
print(f"MOR DELETE 시간: {mor_delete_time:.3f}초")

ratio = cow_delete_time / mor_delete_time if mor_delete_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x")

### 관찰 포인트 — DELETE

- COW: 역시 데이터 파일 재작성
- MOR: Delete File만 추가

---
## 단계 4: SELECT — 읽기 성능 비교

전체 데이터를 읽는 시간을 비교합니다.

In [ ]:
# COW SELECT
start = time.time()
cow_count = spark.sql(f"SELECT COUNT(*) FROM {COW_TABLE}").collect()[0][0]
cow_read_time = time.time() - start
print(f"COW SELECT 시간: {cow_read_time:.3f}초 (레코드 수: {cow_count})")

# MOR SELECT
start = time.time()
mor_count = spark.sql(f"SELECT COUNT(*) FROM {MOR_TABLE}").collect()[0][0]
mor_read_time = time.time() - start
print(f"MOR SELECT 시간: {mor_read_time:.3f}초 (레코드 수: {mor_count})")

ratio = mor_read_time / cow_read_time if cow_read_time > 0 else float('inf')
print(f"\nMOR/COW 비율: {ratio:.2f}x (MOR이 읽기에서 느릴 수 있음)")
print(f"레코드 수 일치 여부: {cow_count == mor_count}")

### 관찰 포인트 — SELECT

- COW: 데이터 파일만 읽으면 됨 → 빠름
- MOR: 데이터 파일 + Delete File 병합 필요 → 상대적으로 느림
- 소규모 데이터에서는 차이가 미미할 수 있지만, 대규모에서 차이가 커집니다

---
## 파일 시스템 비교

In [ ]:
cow_file_count = count_files(COW_PATH)
mor_file_count = count_files(MOR_PATH)
cow_total_size = total_size(COW_PATH)
mor_total_size = total_size(MOR_PATH)

print("=" * 60)
print("COW 테이블 파일 구조:")
print("=" * 60)
show_tree(COW_PATH)

print(f"\n파일 수: {cow_file_count}, 총 크기: {cow_total_size:,} bytes")

print("\n" + "=" * 60)
print("MOR 테이블 파일 구조:")
print("=" * 60)
show_tree(MOR_PATH)

print(f"\n파일 수: {mor_file_count}, 총 크기: {mor_total_size:,} bytes")

---
## 종합 비교표

In [ ]:
comparison = pd.DataFrame({
    '항목': [
        'INSERT 시간 (초)',
        'UPDATE 시간 (초)',
        'DELETE 시간 (초)',
        'SELECT 시간 (초)',
        '파일 수',
        '총 크기 (bytes)',
        '최종 레코드 수',
    ],
    'COW': [
        f"{cow_insert_time:.3f}",
        f"{cow_update_time:.3f}",
        f"{cow_delete_time:.3f}",
        f"{cow_read_time:.3f}",
        cow_file_count,
        f"{cow_total_size:,}",
        cow_count,
    ],
    'MOR': [
        f"{mor_insert_time:.3f}",
        f"{mor_update_time:.3f}",
        f"{mor_delete_time:.3f}",
        f"{mor_read_time:.3f}",
        mor_file_count,
        f"{mor_total_size:,}",
        mor_count,
    ],
})

print("=" * 60)
print("COW vs MOR 종합 비교")
print("=" * 60)
print(comparison.to_string(index=False))

---
## 비교 결과 분석

### 쓰기 성능 (UPDATE/DELETE)
- **MOR이 빠름**: Delete File만 추가하므로 파일 재작성이 없음
- **COW가 느림**: 변경된 행이 포함된 파티션 파일 전체를 재작성

### 읽기 성능 (SELECT)
- **COW가 빠름**: 데이터 파일만 스캔하면 됨
- **MOR이 느림**: 데이터 파일과 Delete File을 병합해야 함

### 파일 수와 크기
- **COW**: 재작성으로 인해 누적 파일 크기가 클 수 있음
- **MOR**: Delete File이 추가로 생성되어 파일 수가 더 많을 수 있음

> **참고**: 소규모 데이터(200건)에서는 차이가 미미할 수 있습니다.  
> 실제 프로덕션(수백만~수십억 건)에서는 이 차이가 **수배~수십배**로 벌어집니다.

---
## 선택 기준 가이드

| 워크로드 | 추천 | 이유 |
|---------|------|------|
| 읽기 중심, 수정 적음 | **COW** | 읽기 최적화, Delete File 없음 |
| 쓰기 중심, 수정 빈번 | **MOR** | 쓰기 빠름, 정기 컴팩션으로 읽기 비용 관리 |
| 혼합 워크로드 | **작업별 혼합 설정** | `write.delete.mode=mor`, `write.update.mode=mor` 등 개별 설정 가능 |

### 실무 팁

- Iceberg에서는 **테이블 속성을 통해 작업별로 다른 전략을 설정**할 수 있습니다
  ```sql
  -- DELETE는 MOR, UPDATE는 COW로 혼합 설정
  ALTER TABLE my_table SET TBLPROPERTIES (
      'write.delete.mode'='merge-on-read',
      'write.update.mode'='copy-on-write'
  )
  ```
- MOR 테이블은 Delete File이 누적되므로, **정기적인 컴팩션(compaction)**이 필요합니다
- 컴팩션은 Delete File을 데이터 파일에 병합하여 읽기 성능을 회복시킵니다

---
## 다음 단계

> MOR 테이블의 읽기 성능을 회복하려면 **정기적인 컴팩션**이 필요합니다.  
> 이 내용은 **4_optimization** 모듈에서 자세히 다룹니다.
>
> 컴팩션을 통해 Delete File을 데이터 파일에 병합하면,  
> MOR의 쓰기 장점을 유지하면서 읽기 성능도 관리할 수 있습니다.

In [ ]:
spark.stop()
print("Spark 세션 종료")